In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from PIL import Image
import os
import datetime
from slackclient import SlackClient
from env import *

def slack_message(message, channel):
    token = SLACK_TOKEN
    sc = SlackClient(token)
    sc.api_call('chat.postMessage', channel=channel, 
                text=message, username='queue_update')

driver = webdriver.Chrome('/Users/jskycak/automation/chromedriver_mac')
driver.get('https://cms8.remilon.com/login')

username = driver.find_element_by_name("username")
password = driver.find_element_by_name("password")

username.send_keys("jskycak")
password.send_keys(CMS_PASS)

driver.find_element_by_class_name("btn").click()

for _ in range(2):
    driver.get('https://cms8.remilon.com/work#!/available')
    time.sleep(5)
    screenshot = driver.save_screenshot('temp/available_items_screenshot.png')
    
    previous_tens = Image.open('temp/available_items_template_tens.png')
    
    img = Image.open('temp/available_items_screenshot.png')
    width, height = img.size[0], img.size[1]
    area_full = (int(0.761*width), int(0.27*height), int(0.78*width), int(0.32*height))
    area_hundreds = (int(0.761*width), int(0.27*height), int(0.768*width), int(0.32*height))
    area_tens = (int(0.767*width), int(0.27*height), int(0.774*width), int(0.32*height))
    area_ones = (int(0.773*width), int(0.27*height), int(0.78*width), int(0.32*height))
    
    img.crop(area_full).save('temp/available_items_template_full.png')
    img.crop(area_ones).save('temp/available_items_template_ones.png')
    img.crop(area_tens).save('temp/available_items_template_tens.png')
    img.crop(area_hundreds).save('temp/available_items_template_hundreds.png')
    
    existing_templates_full = [Image.open('templates/full/'+filename) for filename in os.listdir('templates/full')]
    existing_templates_ones = [Image.open('templates/ones/'+filename) for filename in os.listdir('templates/ones')]
    existing_templates_tens = [Image.open('templates/tens/'+filename) for filename in os.listdir('templates/tens')]
    existing_templates_hundreds = [Image.open('templates/hundreds/'+filename) for filename in os.listdir('templates/hundreds')]

    template_full = Image.open('temp/available_items_template_full.png')
    template_ones = Image.open('temp/available_items_template_ones.png')
    template_tens = Image.open('temp/available_items_template_tens.png')
    template_hundreds = Image.open('temp/available_items_template_hundreds.png')
    
    save_filename = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+".png"
    
    if template_full not in existing_templates_full:
        template_full.save("templates/full/"+save_filename)
    if template_ones not in existing_templates_ones:
        template_ones.save("templates/ones/"+save_filename)
    if template_tens not in existing_templates_tens:
        template_tens.save("templates/tens/"+save_filename)
    if template_hundreds not in existing_templates_hundreds:
        template_hundreds.save("templates/hundreds/"+save_filename)
        
    # if tens place changed since last time, send me an alert
    # figure out how to send image in channel?
    if template_tens != previous_tens:
        slack_message('Queue 10s place has changed','queue_update')

driver.quit()